In [ ]:
!pip install pyowm
from pyowm import OWM
from pyowm.utils import config
from pyowm.utils import timestamps
import json
import requests
from datetime import datetime
from IPython.display import clear_output
import pandas as pd
from datetime import date

clear_output()

#Call Weather API

In [ ]:
def get_weather(city_name, timeslot):
  
  api_key = "b7401d9b77454d7d12cc88ae4af4b510"
  response = requests.get(f"http://api.openweathermap.org/geo/1.0/direct?q={city_name}&limit=5&appid={api_key}")
  data = json.loads(response.text)
  lat = data[0]["lat"]
  lon = data[0]['lon']
  exclude = "minutely,hourly,alerts"
  response = requests.get(f"https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&exclude={exclude}&appid={api_key}&units=metric")
  weather_data = json.loads(response.text)

  # create dataframe 
  daily = weather_data['daily']
  df = pd.DataFrame.from_records(daily)
  df['dt'] = pd.to_datetime(df['dt'],unit='s').dt.normalize()
  df['sunrise'] = pd.to_datetime(df['sunrise'],unit='s')
  df['sunset'] = pd.to_datetime(df['sunset'],unit='s')

  if timeslot == "today":
    df = df.loc[[0]]
    return df

  if timeslot == "tomorrow":
    df = df.loc[[1]].reset_index()
    return df


In [ ]:
# Just for your info of the df
df = get_weather("Mainz", "tomorrow")
df

,index,dt,sunrise,sunset,moonrise,moonset,moon_phase,temp,feels_like,pressure,humidity,dew_point,wind_speed,wind_deg,wind_gust,weather,clouds,pop,uvi
0,1,2021-12-15,2021-12-15 07:19:00,2021-12-15 15:25:16,1639574820,1639538640,0.39,"{'day': 6.42, 'min': 4.36, 'max': 7.29, 'night...","{'day': 6.42, 'night': 4.66, 'eve': 5.56, 'mor...",1033,84,3.51,0.69,176,0.73,"[{'id': 804, 'main': 'Clouds', 'description': ...",94,0,0.21


#Intent functions

In [ ]:
def get_general_weather(location, time):
  if location == '':
    city_name = 'Mannheim'

  else:
    city_name = location

  df = get_weather(city_name, time)
  temp = int(df["temp"][0]["day"])
  feels_like = int(df["feels_like"][0]["day"])
  description = str(df["weather"][0][0]["description"])

  return temp, feels_like, description, city_name

In [ ]:
def get_rain(location, time):
  if location == '':
    city_name = 'Mannheim'

  else:
    city_name = location
  df = get_weather(city_name, time)
  rain_prob = int(df["pop"][0]*100)

  return rain_prob, city_name

In [ ]:
def get_clouds(location, time):
  if location == '':
    city_name = 'Mannheim'

  else:
    city_name = location

  df = get_weather(city_name, time)
  cloudiness = int(df["clouds"][0])

  return cloudiness, city_name

In [ ]:
def get_sunny(location, time):
  if location == '':
    city_name = 'Mannheim'

  else:
    city_name = location

  df = get_weather(city_name, time)
  sunlight = 100-int(df["clouds"][0])
  uv_radiation = df["uvi"][0]

  return sunlight, uv_radiation, city_name

In [ ]:
def get_sun_hours(location, time):
  if location == '':
    city_name = 'Mannheim'

  else:
    city_name = location

  df = get_weather(city_name, time)
  sunrise = str(df["sunrise"][0])[11:]
  sunset = str(df["sunset"][0])[11:]

  return sunrise, sunset, city_name

In [ ]:
def get_temperature(location, time):
  if location == '':
    city_name = 'Mannheim'

  else:
    city_name = location

  df = get_weather(city_name, time)
  temp_day = int(df["temp"][0]["day"])
  temp_night = int(df["temp"][0]["night"])
  temp_eve = int(df["temp"][0]["eve"])

  return temp_day, temp_night, temp_eve, city_name

In [ ]:
def get_wind(location, time):
  if location == '':
    city_name = 'Mannheim'

  else:
    city_name = location

  df = get_weather(city_name, time)
  wind_speed = df["wind_speed"][0]

  return wind_speed, city_name

#Inputs from NLP

In [ ]:
# example inputs
location = 'Wien'
time = 'tomorrow'
intent = 'wind'

#set default time if mo time is given
if time == '':
  time = 'today'

#Outputs for the chatbot

In [ ]:
if intent == 'general_weather':
  temp, feels_like, description, city_name = get_general_weather(location, time)
  output = f"{time.capitalize()} in {city_name} the following is reported: {description.capitalize()}. Temperature is {temp} degrees, it feels like {feels_like} degrees."

if intent == 'rain':
  rain_prob, city_name = get_rain(location, time)
  if rain_prob > 50:
    output = f'The probabilty of {intent} in {city_name} for {time} is: {rain_prob}%. Better take an umbrella with you!'
  else:
    output = f'The probabilty of {intent} in {city_name} for {time} is: {rain_prob}%. Guess you wont need an umbrella!'

elif intent == 'cloud':
  cloudiness, city_name = get_clouds(location, time)
  output = f'The cloud density in {city_name} for {time} is: {cloudiness}%'

elif intent == 'sunny':
  sunlight, uv_radiation, city_name = get_sunny(location, time)
  if uv_radiation > 3:
    output = f"{time.capitalize()} the sun in {city_name} will shine with {sunlight}% and the maximum uv index is {uv_radiation}. I would recommend a sun hat!"
  else:
    output = f"{time.capitalize()} the sun in {city_name} will shine with {sunlight}% and the maximum uv index is {uv_radiation}"
  
elif intent == 'sun_hours':
  sunrise, sunset, city_name = get_sun_hours(location, time)
  output = f'{time.capitalize()} the sun in {city_name} will rise at {sunrise} and set at {sunset} UTC'

elif intent == 'temperature':
  temp_day, temp_night, temp_eve, city_name = get_temperature(location, time)
  if temp_day <= 5:
     output = f"{time.capitalize()} in {city_name} the temperature will be {temp_day}°C during the day, {temp_eve}°C in the evening and {temp_night}°C at night. I would recommend to wear a scarf!"
  else:
    output = f"{time.capitalize()} in {city_name} the temperature will be {temp_day}°C during the day, {temp_eve}°C in the evening and {temp_night}°C at night"

elif intent == 'wind':
  wind_speed, city_name = get_wind(location, time)
  output = f'{time.capitalize()} the wind in {city_name} will be {wind_speed} km/h fast.'

elif intent == 'good_bad':
  output = "Grillenziepen"

print(output)

Tomorrow the wind in Wien will be 6.9 km/h fast.
